# USC CODE START

In [2]:
import pandas as pd
import geopandas  as gpd
import os
from sqlalchemy import engine
from PIL import Image
from PIL.ExifTags import TAGS
from datetime import datetime
from sqlalchemy import create_engine
import rasterio as rio
from rasterio.features import dataset_features
from osgeo import gdal
import psycopg2
from shapely import wkt
from shapely.validation import make_valid
import glob
import subprocess
import tempfile
import time

In [3]:
def humanbytes(B):
    """Return the given bytes as a human friendly KB, MB, GB, or TB string."""
    B = float(B)
    KB = float(1024)
    MB = float(KB ** 2) # 1,048,576
    GB = float(KB ** 3) # 1,073,741,824
    TB = float(KB ** 4) # 1,099,511,627,776

    if B < KB:
        return '{0} {1}'.format(B,'Bytes' if 0 == B > 1 else 'Byte')
    elif KB <= B < MB:
        return '{0:.2f} KB'.format(B / KB)
    elif MB <= B < GB:
        return '{0:.2f} MB'.format(B / MB)
    elif GB <= B < TB:
        return '{0:.2f} GB'.format(B / GB)
    elif TB <= B:
        return '{0:.2f} TB'.format(B / TB)

In [4]:
def mk_coverage(env, img_path, cov_path):
    os.chdir(env)
    cmd = f'gdaltindex {cov_path} {img_path}'
    subprocess.call(cmd,shell=True)
    # print(f'coverage created -- {cov_path}')
    return(cov_path)

## FUNC COVERAGES

In [5]:
folder = 'D:/County_fips/fips_year'
env = 'C:/Users/terra/anaconda3/envs/pygdal'

In [5]:
# env = 'C:\\Users\\terra\\anaconda3\\envs\\pygdal'

def create_coverages(folder):
    # ------------------------------------------------- #
    dfdict = {'img_name':[], 'fips':[],'dt_year':[],'img_num':[], 'img_size':[],
            'meta_size':[],'meta_unit':[],'state':[],'meta_last_acc':[],'img_path':[]}
    # ------------------------------------------------- #
    dfdict_cov_geom = {'img_name':[], 'area_sqkm':[], 'shape':[]}
    # ------------------------------------------------- #
    for image in glob.glob(os.path.join(folder, '*.tif')):
        image = image.replace('\\','/')
        # ------------------------------------------------- #
        img_path    = os.path.join(folder, image)
        stats       = os.stat(img_path)
        # temp path for coverage geojson
        # create a temporary folder using library tempfile
        temp_dir = tempfile.mkdtemp()
        temp_cov_path = os.path.join(temp_dir, 'temp_coverage.geojson')
        # ------------------------------------------------- #
        # get the image name without extension
        img_name = image.split('\\')[-1].replace('.tif','')
        # img_name    = os.path.splitext(image)[0] ; print(img_name)
        imgsplit    = img_name.split('_')
        # ------------------------------------------------- #
        (fips, dt_year, img_num, img_size,
            meta_size, meta_lastmod, meta_lastacc) = (imgsplit[0], imgsplit[1],
                                                    imgsplit[3], imgsplit[5], humanbytes(round(stats.st_size)),
                                                        datetime.fromtimestamp(stats.st_mtime),
                                                        datetime.fromtimestamp(stats.st_atime))
        # ------------------------------------------------- #
        cov_path        = mk_coverage(env, img_path, temp_cov_path)
        crs_string      = 'EPSG:3857'
        gdf             = gpd.read_file(cov_path, crs=crs_string)
        gdf             = gdf.dissolve()
        gdf['geometry'] = gdf['geometry'].apply(make_valid)
        gdf['area_sqkm']= gdf['geometry'].map(lambda p: p.area / 10**6)
        sqkm            = int(round(sum(gdf.area_sqkm),2))
        geom            = gdf['geometry'].values[0]
        # ------------------------------------------------- #
        dfdict['img_name'].append(str(img_name))      ;  dfdict['fips'].append(str(fips)),
        dfdict['dt_year'].append(str(dt_year))        ;  dfdict['img_num'].append(str(img_num)),
        dfdict['img_size'].append(str(img_size))      ;  dfdict['meta_size'].append(str(meta_size).replace(' MB',''))
        dfdict['meta_last_acc'].append(str(meta_lastacc))  ;  dfdict['img_path'].append(str(img_path))
        dfdict['meta_unit'].append('MB')                   ;  dfdict['state'].append('South Carolina')
        # ------------------------------------------------- #
        dfdict_cov_geom['img_name'].append(str(img_name))
        dfdict_cov_geom['area_sqkm'].append(sqkm)#(int(round(sum(gdf.area_sqkm),2)))
        dfdict_cov_geom['shape'].append(geom)
    # merge both dictionaries on img_name
    merge_dict = {**dfdict, **dfdict_cov_geom}
    return(pd.DataFrame(merge_dict))





In [6]:
df  = create_coverages(folder)
gdf = gpd.GeoDataFrame(df, geometry=df['shape'])
gdf = gdf.drop(columns=['shape'])
gdf.to_file(os.path.join(folder,'cov/coverages.geojson'), driver="GeoJSON") 

## FUNC 8BIT MOSAIC UPDATE

In [31]:

def tiff_to_tile(folder):
    bit_webp_ls     = []
    need_process    = []
    txt_ls          = []
    # -------------------- #
    # create a temp directory to store 8bit.webp files
    temp_dir = tempfile.mkdtemp()
    # -------------------- #
    # list of tiff files
    tiff_ls = [t.replace('\\','/') for t in glob.glob(os.path.join(folder, '*.tif'))]
    tiff_ls = [t.split('/')[-1].replace('.tif','') for t in tiff_ls]
    # -------------------- #
    
    # read list from text file
    with open(os.path.join(folder,'cov_8bit/bit_webp_ls.txt'), 'r') as f:
        for line in f:
            txt_ls.append(line.split(',')) 
    # flatten list
    txt_ls = [item for sublist in txt_ls for item in sublist]  
    # print(f'list from text file: {txt_ls}')
    # print(f'list of tiff files: {tiff_ls}')
    
    # -------------------- #
    
    # if tiff names are not in txt file, add to need_process list
    if txt_ls not in tiff_ls:
        # append list of tiff files that are not in txt file
        need_process.append([x for x in tiff_ls if x not in txt_ls])
    # flatten list
    need_process = [item for sublist in need_process for item in sublist]
    # print(f'list of tiff files that need to be processing: {need_process}')
    
    # -------------------- #
    
    for image in need_process: 
        # folders and files var establishment
        img_name = image.split('/')[-1].replace('.tif','')
        org_tif  = os.path.join(folder, f'{img_name}.tif')
        bit_webp   = os.path.join(temp_dir, f'{img_name}_8bit.webp')
        # -------------------- #
        # print(f'processing {img_name}...')
        # convert tiff to 8bit webp format
        cmd_8bit = f'gdal_translate -ot Byte -scale {org_tif} {bit_webp}'
        # converte tiff to 8bit webp format
        subprocess.call(cmd_8bit,shell=True)
        # append to list of 8bit.webp files
        bit_webp_ls.append(img_name)  
        # print(f'{img_name} completed')
    
    # -------------------- #
    
    # overwrite current bit_webp_ls.txt & save list of 8bit.webp files to a text file for future use
    with open(os.path.join(folder,'cov_8bit/bit_webp_ls.txt'), 'w') as f:
        # write each item from the list to text file
        for item in bit_webp_ls:
            f.write("%s" % item)
    print(f'list of 8bit.webp files: {bit_webp_ls}') 
    ## SOLVE FOR REFRESHING THE LIST OF 8BIT.WEBP FILES ##
    
    ## FINAL VERSIION NEEDS VARS FOR 3 IN-YEAR FOLDERS ##
    
    # -------------------- #
    # mosaic all 8bit.webp files in temp_dir
    
    # list of all 8bit.webp files in temp_dir
    webp_files = glob.glob(os.path.join(temp_dir, '*.webp'))
    tile_folder = os.path.join(folder,'mosaic').replace('\\','/')
    # gdal mosaic all files in temp folder
    cmd_mosaic = f'''gdalbuildvrt -overwrite -resolution average
     -r nearest -input_file_list {webp_files} {tile_folder}/mosaic.vrt'''
    subprocess.call(cmd_mosaic,shell=True) 
    print(f'mosaic.vrt created\n{tile_folder}')
    # print(webp_files)
    
    ## TROUBLESHOOT WHY gdalbuildvrt IS NOT WORKING ##
    
    
    
    
tiff_to_tile(folder)

list of 8bit.webp files: ['45085_1980_179_0001_x_9', '45085_1980_179_0005_x_9', '45085_1980_179_0003_x_9']
mosaic.vrt created
D:/County_fips/fips_year/mosaic
